In [ ]:
pip install azureml-sdk

In [ ]:
from azureml.core import Workspace, Datastore, Dataset

Workspace

In [ ]:
#Create Workspace
ws = Workspace.create(name='Azureml-SDK-WS01',
                      subscription_id='12345a67b-1357c2468d',
                      resource_group='AZUREMLSDKG01',
                      create_resource_group=True,
                      location='westus')

In [ ]:
#Configure Workspace
ws.write_config(path="./config")

Datastore

In [ ]:
#Access Workspace
ws = Workspace.from_config(path="./config")

In [ ]:
#Create Datastore
az_store = Datastore.register_azure_blob_container(
            workspace=ws,
            datastore_name="azure_sdk_blob01",
            account_name="azuremlstb01",
            container_name="azuremlstb01blob",
            account_key="mQ6meDug7SdlFXu0/tBu7pKcNerxxYtO6X1V13M4sSohBAv2/i2KxdYe3ueiQXKrw+alPU1ou4NBuYBtuBVsig==")

Dataset

In [ ]:
# Create the path of the csv file
csv_path = [(az_store, "Loan Data/Loan Approval Prediction.csv")]

In [ ]:
# Create the dataset
loan_dataset = Dataset.Tabular.from_delimited_files(path=csv_path)

In [ ]:
# Register the dataset
loan_dataset = loan_dataset.register(workspace=ws,
                                     name="Loan Applications Using SDK",
                                     create_new_version=True)

Access Workspace, Datastore, Dataset from SDK

In [ ]:
# List all the workspaces within a subscription
ws_list = Workspace.list(subscription_id="77819c59-5764-4995-8596-d09cdc661078")
ws_list = list(ws_list)

In [ ]:
# Access the default datastore from workspace
az_default_store = ws.get_default_datastore()

In [ ]:
# List all the datastores
store_list = list(ws.datastores)

In [ ]:
# Get the dataset by name from a workspace
az_dataset = Dataset.get_by_name(ws, "Loan Applications Using SDK")

In [ ]:
# List datasets from a workspace
ds_list = list(ws.datasets.keys())

for items in ds_list:
    print(items)

Pandas Dataframe and AzureML Dataset

In [ ]:
# Access the Workspace, Datastore and Datasets
ws                = Workspace.from_config("./config")
az_store          = Datastore.get(ws, 'azure_sdk_blob01')
az_dataset        = Dataset.get_by_name(ws, 'Loan Applications Using SDK')
az_default_store  = ws.get_default_datastore()

In [ ]:
# Load the Azureml Dataset into the pandas dataframe
df = az_dataset.to_pandas_dataframe()

In [ ]:
# Upload the dataframe to the azureml dataset
df_sub = df[["Married", "Gender", "Loan_Status"]]

az_ds_from_df = Dataset.Tabular.register_pandas_dataframe(
                dataframe=df_sub,
                target=az_store,
                name="Loan Dataset From Dataframe")

Upload local data to storage account via datastore

In [ ]:
# Upload local files to storage account using datastore 
files_list = ["./data/test.csv", "./data/test1.csv"]

az_store.upload_files(files=files_list,
                      target_path="Loan Data/",
                      relative_root="./data/",
                      overwrite=True)

In [ ]:
# Upload folder or directory to the storage account
az_store.upload(src_dir="./data",
                target_path="Loan Data/data",
                overwrite=True)